In [ ]:
from google.colab import files
uploaded = files.upload()

Saving images.zip to images.zip


In [ ]:
import zipfile
import os

# Replace 'your_folder.zip' with your zip file's name
zip_ref = zipfile.ZipFile('/content/images.zip', 'r')
zip_ref.extractall('images')  # or just extractall() to current directory
zip_ref.close()


In [ ]:
!pip install keras_preprocessing

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 4.3 MB/s eta 0:00:00


In [ ]:
from keras.utils import to_categorical
from keras_preprocessing.image import load_img
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
import os
import pandas as pd
import numpy as np

In [ ]:
TRAIN_DIR = '/content/images/images/train'
TEST_DIR = '/content/images/images/test'

In [ ]:
def createdataframe(dir):
    image_paths = []
    labels = []
    for label in os.listdir(dir):
        for imagename in os.listdir(os.path.join(dir,label)):
            image_paths.append(os.path.join(dir,label,imagename))
            labels.append(label)
        print(label, "completed")
    return image_paths,labels

In [ ]:
train = pd.DataFrame()
train['image'], train['label'] = createdataframe(TRAIN_DIR)

fear completed
neutral completed
surprise completed
happy completed
angry completed
disgust completed
sad completed


In [ ]:
test = pd.DataFrame()
test['image'], test['label'] = createdataframe(TEST_DIR)

fear completed
neutral completed
surprise completed
happy completed
angry completed
disgust completed
sad completed


In [ ]:
from tqdm.notebook import tqdm

In [ ]:
def extract_features(images):
    features = []
    for image in tqdm(images):
        img = load_img(image, color_mode="grayscale")
        img = np.array(img)
        features.append(img)
    features = np.array(features)
    features = features.reshape(len(features),48,48,1)
    return features

In [ ]:
train_features = extract_features(train['image'])

  0%|          | 0/28821 [00:00<?, ?it/s]

In [ ]:
test_features = extract_features(test['image'])

  0%|          | 0/7066 [00:00<?, ?it/s]

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
le = LabelEncoder()
le.fit(train['label'])


LabelEncoder()

In [ ]:
y_train = le.transform(train['label'])
y_test = le.transform(test['label'])

In [ ]:
y_train = to_categorical(y_train,num_classes = 7)
y_test = to_categorical(y_test,num_classes = 7)

In [ ]:
model = Sequential()
# convolutional layers
model.add(Conv2D(128, kernel_size=(3,3), activation='selu', input_shape=(48,48,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(256, kernel_size=(3,3), activation='selu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='selu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='selu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
# fully connected layers
model.add(Dense(512, activation='selu'))
model.add(Dropout(0.4))
model.add(Dense(256, activation='selu'))
model.add(Dropout(0.3))
# output layer
model.add(Dense(7, activation='softmax'))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
x_train = train_features/255.0
x_test = test_features/255.0

In [ ]:
model.fit(x= x_train,y = y_train, batch_size = 128, epochs = 100, validation_data = (x_test,y_test))

Epoch 1/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 27s 81ms/step - accuracy: 0.2050 - loss: 3.0118 - val_accuracy: 0.3818 - val_loss: 1.6009
Epoch 2/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 29s 52ms/step - accuracy: 0.3541 - loss: 1.6498 - val_accuracy: 0.4295 - val_loss: 1.5725
Epoch 3/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 12s 55ms/step - accuracy: 0.3990 - loss: 1.5720 - val_accuracy: 0.4624 - val_loss: 1.4019
Epoch 4/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 20s 53ms/step - accuracy: 0.4215 - loss: 1.5045 - val_accuracy: 0.4809 - val_loss: 1.4243
Epoch 5/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 20s 53ms/step - accuracy: 0.4479 - loss: 1.4507 - val_accuracy: 0.5228 - val_loss: 1.2926
Epoch 6/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 21s 53ms/step - accuracy: 0.4712 - loss: 1.3871 - val_accuracy: 0.5340 - val_loss: 1.2397
Epoch 7/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 20s 53ms/step - accuracy: 0.4989 - loss: 1.3146 - val_accuracy: 0.5335 - val_loss: 1.2478
Epoch 8/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 12s 53ms/step - accuracy: 0.5120 - loss: 1

In [ ]:
label = ['angry','disgust','fear','happy','neutral','sad','surprise']

In [ ]:
def ef(image):
    img = load_img(image,grayscale =  True )
    feature = np.array(img)
    feature = feature.reshape(1,48,48,1)
    return feature/255.0


In [ ]:
image = "/content/images/images/test/surprise/10097.jpg"
img = ef(image)
pred = model.predict(img)
pred_label = label[pred.argmax()]
pred_label

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step


'neutral'

In [ ]:
import numpy as np

# Get predictions
y_pred = model.predict(x_test)

# Convert predictions to class labels
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

# Calculate accuracy
accuracy_manual = np.mean(y_pred_classes == y_true_classes)

221/221 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step


In [ ]:
print(accuracy_manual)

0.6303424851401076


In [ ]:
model.save('mymodel.h5')